Max3-SAT_to_QUBO

Construct QUBO matrix for 3SAT
From:
$$
-g(x,w) = K + [x, w]^T F(phi) [x, w] 
$$

Ex.
$$
-g(x, w) = -4 + 0x_1 - 2x_1x_2 + 2x_1x_3 - x_1w_1 + x_1w_2 - x_1w_3 + x_1w_4 \\
+ x_2 - 0x_2x_3 - x_2w_1 - x_2w_2 + x_2w_3 - x_2w_4 - x_3 - x_3w_1 - x_3w_2 \\
- x_3w_3 + x_3w_4 + 2w_1 + 0(w_1w_2 + w_1w_3 + w_1w_4) \\
+ w_2 + 0(w_2w_3 + w_2w_4) + w_3 + 0w_3w_4 + 0w_4\\
$$\\


Ex. $$φ = (x1 ∨ x2 ∨ x3) ∧ (-x1 ∨ x2 ∨ x3) ∧ (x1 ∨ -x2 ∨ x3) ∧ (-x1 ∨ x2 ∨ -x3)$$

In [178]:
import numpy as np
import sympy
import re

In [179]:
num_x = 3
num_m = 4
phi_matrix = np.zeros((num_m, num_x), dtype=int)
display(phi_matrix) # phi_matrix[clause][x]

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

x presents in each clause

In [180]:
phi_matrix[0] = 1
phi_matrix[1] = 1
phi_matrix[2] = 1
phi_matrix[3] = 1
display(phi_matrix)

array([[1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1]])

not x in each clause

In [181]:
phi_matrix[1][0] = -1
phi_matrix[2][1] = -1
phi_matrix[3][0] = -1
phi_matrix[3][2] = -1
display(phi_matrix)

array([[ 1,  1,  1],
       [-1,  1,  1],
       [ 1, -1,  1],
       [-1,  1, -1]])

In [182]:
def problem_3SAT_to_QUBO(phi_matrix):
    num_x = phi_matrix.shape[1] # Phi matrix Column
    x = sympy.symbols(f'x0:{num_x}') # x0, x1, ... x_num_x -1
    num_m = phi_matrix.shape[0] # Phi matrix Row
    w = sympy.symbols(f'w0:{num_m}') # w0, w1, ... w_num_m - 1
    QUBO_matrix = np.zeros((num_x + num_m, num_x + num_m), dtype=int)
    sum_g = 0

    error = False
    
    for i in range(num_m):
        x_array = phi_matrix[i]
        x_presents_indices = np.where(x_array != 0)[0]
        # Make sure each clause has only 3 literals
        if (x_presents_indices.shape[0] != 3):
            error = True
            break

        y_i1 = x[x_presents_indices[0]] if (x_array[x_presents_indices[0]] == 1) else (1 + (-1 * x[x_presents_indices[0]]))
        y_i2 = x[x_presents_indices[1]] if (x_array[x_presents_indices[1]] == 1) else (1 + (-1 * x[x_presents_indices[1]]))
        y_i3 = x[x_presents_indices[2]] if (x_array[x_presents_indices[2]] == 1) else (1 + (-1 * x[x_presents_indices[2]]))
        sum_g += y_i1 + y_i2 + y_i3 + (w[i] * y_i1) + (w[i] * y_i2) + (w[i] * y_i3) - (y_i1 * y_i2) - (y_i1 * y_i3) - (y_i2 * y_i3) - 2 * w[i]
    
    sum_neg_g = sympy.simplify(-1 * sum_g)
    sum_neg_g_dict = {str(term): coefficient for term, coefficient in sum_neg_g.as_coefficients_dict().items()}

    # for i in range(num_x + num_m):
    #     for j in range(i, num_x + num_m):
    #             if (i == j and i >= num_x): # Skip w[i] * w[j] (These terms do not appear), Only consider w[i==j]
    #                 QUBO_matrix[i][j] = 1
    #                 break
    #             elif (i == j): # x[i]
    #                 QUBO_matrix[i][j] = 2
    #             elif (j < num_x): # x[i] * x[j]
    #                 QUBO_matrix[i][j] = 3
    #             elif (j >= num_x): # x[i] * w[j]
    #                 QUBO_matrix[i][j] = 4

    for term in sum_neg_g_dict:
        # print(f'{term}: {sum_neg_g_dict[term]}')
        if re.match(r'^w\d+$', term): # w[i]
            i = int(term[1:]) + num_x
            j = int(term[1:]) + num_x
            QUBO_matrix[i][j] = sum_neg_g_dict[term]
        elif re.match(r'^x\d+$', term): # x[i]
            i = int(term[1:])
            j = int(term[1:])
            QUBO_matrix[i][j] = sum_neg_g_dict[term]
        elif re.match(r'^w(0|[1-9][0-9]*)\*x(\d+)$', term): # w[i] * x[j]
            match = re.match(r'^w(0|[1-9][0-9]*)\*x(\d+)$', term)
            w_number = int(match.group(1))
            x_number = int(match.group(2))
            QUBO_matrix[x_number][w_number + num_x] = sum_neg_g_dict[term]
        elif re.match(r'^x(0|[1-9][0-9]*)\*x(\d+)$', term): # x[i] * x[j]
            match = re.match(r'^x(0|[1-9][0-9]*)\*x(\d+)$', term)
            first_x_number = int(match.group(1))
            second_x_number = int(match.group(2))
            QUBO_matrix[first_x_number][second_x_number] = sum_neg_g_dict[term]
    
    if (error):
        return "An error has occured, Make sure phi_matrix is of 3SAT kind (Only 3 literals in each clauses)"
    else:
        return QUBO_matrix

In [183]:
print(problem_3SAT_to_QUBO(phi_matrix))

[[ 0 -2  2 -1  1 -1  1]
 [ 0  1  0 -1 -1  1 -1]
 [ 0  0 -1 -1 -1 -1  1]
 [ 0  0  0  2  0  0  0]
 [ 0  0  0  0  1  0  0]
 [ 0  0  0  0  0  1  0]
 [ 0  0  0  0  0  0  0]]


Solve for QUBO(F(phi)) -> min [x, w]^T F(phi) [x, w]